# **Google Drive연결 및 Module 불러오기**

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!nvidia-smi

Mon Nov 21 15:06:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **PM2.5 with Covariate 데이터 불러오기 및 전처리**

In [ ]:
df = pd.read_csv('PM25_with_covariate_220201_ver2.csv')
df.head(6)

,code,x,y,t,z,tmp,uwind,vwind,wind,hum,prec
0,1,126.9747,37.5643,1,27,-0.127208,0.210263,-1.678297,0.608896,79.262265,0.042478
1,1,126.9747,37.5643,2,19,-0.747990,0.257965,-1.256667,0.386535,72.087742,0.038822
2,1,126.9747,37.5643,3,11,-0.793291,0.297994,-1.313214,0.493171,71.505221,0.020449
3,1,126.9747,37.5643,4,9,-1.753191,0.215887,-0.811954,0.243450,84.203073,0.051435
4,1,126.9747,37.5643,5,9,-2.032537,0.105124,-0.322796,-0.157497,87.085583,0.120312
5,1,126.9747,37.5643,6,7,-1.859001,0.054214,-0.760730,0.265878,88.018236,0.080176


In [ ]:
# Target data processing
df_pm25 = df['z']
pm25 = df_pm25.values

z = pm25[:,None]

In [ ]:
# Lon and Lat data normalizing and processing
lon = df.values[:,1]
lat = df.values[:,2]

normalized_lon = (lon-min(lon))/(max(lon)-min(lon))
normalized_lat = (lat-min(lat))/(max(lat)-min(lat))


# Input
df["x"] = normalized_lon
df["y"] = normalized_lat


# Extraction
aqs_lonlat=df.values[:,[1,2]]
print(aqs_lonlat.shape)

(10704, 2)


In [ ]:
N = lon.shape[0]
print(N)

10704


# **Make sptio-temporal basis function**

In [ ]:
# np.array 모두 보기
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
#########################################################
# make sptio-temporal basis function
num_basis = [10**2, 19**2, 37**2 ]

knots_1dt = [ np.linspace( 0,25, int(i) ) for i in num_basis ] #1차원이므로 루트 없앰
knots_1dx = [ np.linspace(0,1, int(np.sqrt(i)) ) for i in num_basis ] #error
knots_1dy = [ np.linspace(0,1, int(np.sqrt(i)) ) for i in num_basis ] 

phi = np.zeros((N, sum(num_basis))) 
phi.shape

(10704, 1830)

In [ ]:
T = np.linspace( 1, 24, 24) 
T = np.tile(T, int(N/24))

#########################################################
#################### Wendland kernel ####################
basis_size = 0


for res in range(len(num_basis)):
    theta = 1 #/ np.sqrt(num_basis[res])
    
    knots_t = knots_1dt[res]
    
    knots_x, knots_y = np.meshgrid( knots_1dx[res] , knots_1dy[res] ) ###
    knots = np.column_stack((knots_x.flatten(),knots_y.flatten()))
        
    if res == 0:
        for i in range(num_basis[res]): # 100개 #가로
            d = abs( T - np.linspace(knots_t[i], knots_t[i], N)  ) / theta # 1-Dimension #make
        
            for j in range(len(d)): # j = 1, \cdots, 24 #세로 
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + basis_size] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + basis_size] = 0
                    
        basis_size += num_basis[res]              
            
        
    else: 
        
        for i in range(num_basis[res]):
            d = np.linalg.norm(np.vstack( (normalized_lon, normalized_lat) ).T - knots[i,:],axis=1)/theta
            
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + basis_size] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + basis_size] = 0  
                
        basis_size = basis_size + num_basis[res]    


    
## Romove the all-zero columns
idx_zero = np.array([], dtype=int)

for i in range(phi.shape[1]):
    if sum(phi[:,i]!=0)==0:
        idx_zero = np.append(idx_zero,int(i))
        
phi_reduce = np.delete(phi,idx_zero,1)
print(phi.shape)
print(phi_reduce.shape) # zero reduced

(10704, 1830)
(10704, 1828)


# **Make Covariate**

In [ ]:
df.iloc[[0,1],[5, 6, 7, 8, 9, 10]]

,tmp,uwind,vwind,wind,hum,prec
0,-0.127208,0.210263,-1.678297,0.608896,79.262265,0.042478
1,-0.747990,0.257965,-1.256667,0.386535,72.087742,0.038822


In [ ]:
covariates = df.values[:,[1, 2, 5, 6, 7, 8, 9, 10]]
covariates.shape

(10704, 8)

In [ ]:
phi_obs = phi #v
 
#~~

s_obs = np.vstack((normalized_lon, normalized_lat)).T #v

#~~

X = covariates
normalized_X = X #v

for i in range(X.shape[1]):
    normalized_X[:,i] = (X[:,i]-min(X[:,i]))/(max(X[:,i])-min(X[:,i]))

N_obs = X.shape[0]

print(N_obs)

10704


# **Make Ensemble DNN Model Function**

In [ ]:
# Function for Model Compile, Fit, Test, Prediction
def deep_model(model, X_train, y_train, X_test, y_test):

    model.compile(optimizer='adam', loss='mse', metrics=['mse','mae'])

    model.fit(X_train, y_train, epochs = NB_START_EPOCHS, batch_size = BATCH_SIZE, verbose=0)
    
    results = model.evaluate(X_test, y_test, verbose=0)
    
    train_prediction = model.predict(X_train, verbose = 0)
    test_prediction = model.predict(X_test, verbose = 0)

    return train_prediction, test_prediction, results

In [ ]:
# Build Basis Version DNN Model
def build_model1():
  p1 = phi_reduce.shape[1]
  p2 = s_obs.shape[1]
  p = p1 + p2

  n = p
  m = 1


  model = Sequential()
  model.add(Dense(p+1+2, input_dim = p,  kernel_initializer='he_uniform', activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(BatchNormalization())

  model.add(Dense(p+1+2, activation='relu'))
  model.add(Dropout(rate=0.5))

  model.add(Dense(p+1+2, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='linear'))
  
  return model

# Build Covariate Version DNN Model => regression으로 대체하자
def build_model2():
  p1 = covariates.shape[1]
  p2 = s_obs.shape[1]
  p = p1 + p2

  model = Sequential()
  model.add(Dense(p+1+2, input_dim = p,  kernel_initializer='he_uniform', activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(BatchNormalization())

  model.add(Dense(p+1+2, activation='relu'))
  model.add(Dropout(rate=0.5))

  model.add(Dense(p+1+2, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='linear'))

  return model


# Build Ensemble Version DNN Model
def build_model3():
  model = Sequential()
  model.add(Dense(10, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation = 'linear'))
  return model  

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
import random

In [ ]:
# Make the inputs and targets
inputs1 = np.hstack((phi_reduce , s_obs))
inputs2 = np.hstack((s_obs, normalized_X))

targets = z

NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
BATCH_SIZE = 64  # Size of the batches used in the mini-batch gradient descent

# **교차검증**

In [ ]:
date = ["01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00",
        "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00", "00:00"]

loc1 = loc2 = loc3 = loc4 = loc5 = []
time1 = time2 = time3 = time4 = time5 = []
location = [ loc1, loc2, loc3, loc4, loc5 ]
time = [time1, time2, time3, time4, time5]

In [ ]:
for i in range(5):
  print("-----------------------------------------")
  j = i + 1
  random.seed(j*10)
  
  time[i] = sorted(random.sample(range(24), 2))
  location[i] = sorted(random.sample(range(446), 30))
  
  print("Test location",i+1,": ",location[i])
  print("Test time",i+1,": ",time[i])
  print("Test time",i+1,": ",date[ time[i][0] - 1 ], "/" , date[ time[i][1] - 1])

-----------------------------------------
Test location 1 :  [7, 17, 22, 38, 71, 82, 105, 127, 142, 145, 167, 181, 184, 195, 215, 219, 236, 247, 250, 251, 266, 295, 308, 334, 381, 415, 416, 421, 423, 440]
Test time 1 :  [1, 18]
Test time 1 :  01:00 / 18:00
-----------------------------------------
Test location 2 :  [13, 38, 47, 51, 52, 64, 77, 86, 102, 106, 133, 162, 163, 167, 168, 171, 208, 210, 218, 230, 242, 293, 297, 320, 325, 345, 348, 392, 402, 434]
Test time 2 :  [21, 23]
Test time 2 :  21:00 / 23:00
-----------------------------------------
Test location 3 :  [3, 13, 15, 24, 34, 37, 41, 68, 81, 107, 124, 131, 178, 192, 203, 205, 236, 267, 271, 274, 306, 312, 318, 328, 334, 335, 417, 423, 425, 431]
Test time 3 :  [9, 17]
Test time 3 :  09:00 / 17:00
-----------------------------------------
Test location 4 :  [14, 16, 27, 30, 65, 66, 90, 103, 105, 125, 141, 144, 160, 178, 225, 235, 268, 270, 309, 317, 326, 328, 340, 362, 377, 380, 390, 405, 419, 435]
Test time 4 :  [14, 18]
Tes

In [ ]:
i=1
j=1
k=1

MSE_Basis = []
MSE_Covariate = []
MSE_Covariate_Reg = []
MSE_DNN_DNN = []
MSE_Reg_DNN = []
MSE_Reg_Wgt = []
MSE_DNN_Wgt = []

for test_time in time:

  for test_loc in location:

      print('------------------------------------------------------------------------')
      print('Test for time = ',i, ' location = ',j,'... (', k , '/', len(time)*len(location), ')' )

      index1 = df[df['t'].isin( test_time )].index
      index2 = df[df['code'].isin( test_loc )].index

      train_idx_inv = list(set(index1) | set(index2)) 
      train_idx = list(set(df.index) - set(train_idx_inv))
      test_idx = list(set(index1) & set(index2))



      model1 = build_model1()
      model2 = build_model2()
      model3 = build_model3()

      #------

      train_pred1, test_pred1, result1 = deep_model(model1, inputs1[train_idx], targets[train_idx], inputs1[test_idx], targets[test_idx])

      print("model1 (DNN with Basis) fit..", end = "")
      print(f'MSE = {result1[1]},,, MAE = {result1[2]}')
      MSE_Basis.append(result1[1]) ##save

      train_pred2, test_pred2, result2 = deep_model(model2, inputs2[train_idx], targets[train_idx],inputs2[test_idx], targets[test_idx])

      print("model2 (DNN with Covariate) fit..", end = "")
      print(f'MSE = {result2[1]},,, MAE = {result2[2]}')
      MSE_Covariate.append(result2[1])


      mlr = LinearRegression()
      mlr.fit(inputs2[train_idx], targets[train_idx]) 
      MSE_reg = sum((targets[test_idx] - mlr.predict(inputs2[test_idx]))**2)/ len(test_idx)
      MAE_reg = sum( np.abs( targets[test_idx] - mlr.predict(inputs2[test_idx] ) ) )/ len(test_idx)
      train_pred_reg = mlr.predict(inputs2[train_idx]) 
      test_pred_reg = mlr.predict(inputs2[test_idx])

      print("model_reg (Regression with Covariate) fit..", end = "")
      print(f'MSE = {MSE_reg[0]},,, MAE = {MAE_reg[0]}')
      MSE_Covariate_Reg.append(MSE_reg)

      #------

      X_train_new = np.hstack( ( train_pred1, train_pred2  )  )
      X_test_new = np.hstack( ( test_pred1, test_pred2  )  )

      X_train_new2 = np.hstack( ( train_pred1, train_pred_reg  )  )
      X_test_new2 = np.hstack( ( test_pred1, test_pred_reg  )  )


      train_pred3, test_pred3, result3 = deep_model(model3, X_train_new, targets[train_idx], X_test_new, targets[test_idx])

      train_pred_reg, test_pred_reg, result_reg = deep_model(model3, X_train_new2, targets[train_idx], X_test_new2, targets[test_idx])


      stacker = LinearRegression()
      stacker.fit(X = X_train_new, y = targets[train_idx])
      MSE_Wgt1 = np.sum((stacker.predict(X_test_new) - targets[test_idx])**2) / len(test_idx)
      MAE_Wgt1 = np.sum( np.abs(stacker.predict(X_test_new) - targets[test_idx])) / len(test_idx)

      stacker.fit(X = X_train_new2, y = targets[train_idx])
      MSE_Wgt2 = np.sum((stacker.predict(X_test_new2) - targets[test_idx])**2) / len(test_idx)
      MAE_Wgt2 = np.sum( np.abs(stacker.predict(X_test_new2) - targets[test_idx])) / len(test_idx)


      print(f'-> The performance: MSE = {result3[1]}, MAE = {result3[2]} (DNN + DNN)')
      print(f'-> The performance: MSE = {result_reg[1]}, MAE = {result_reg[2]} (Reg + DNN)')
      MSE_DNN_DNN.append(result3[1])
      MSE_Reg_DNN.append(result_reg[1])




      print(f'-> The performance: MSE = {MSE_Wgt1}, MAE = {MAE_Wgt1} (DNN + Wgt)')
      print(f'-> The performance: MSE = {MSE_Wgt2}, MAE = {MAE_Wgt2} (Reg + Wgt)')
      MSE_DNN_Wgt.append(MSE_Wgt1)
      MSE_Reg_Wgt.append(MSE_Wgt2)



      k = k + 1
      j = j + 1
    
  i = i + 1
  j = 1

------------------------------------------------------------------------
Test for time =  1  location =  1 ... ( 1 / 25 )
model1 (DNN with Basis) fit..MSE = 47.361324310302734,,, MAE = 5.396949291229248
model2 (DNN with Covariate) fit..MSE = 68.0084457397461,,, MAE = 6.49793815612793
model_reg (Regression with Covariate) fit..MSE = 89.69480496436695,,, MAE = 7.283934983610507
-> The performance: MSE = 38.534568786621094, MAE = 4.55187463760376 (DNN + DNN)
-> The performance: MSE = 42.27595520019531, MAE = 4.875837326049805 (Reg + DNN)
-> The performance: MSE = 37.98850121400107, MAE = 4.5239639600118 (DNN + Wgt)
-> The performance: MSE = 38.20030614669864, MAE = 4.492674681735908 (Reg + Wgt)
------------------------------------------------------------------------
Test for time =  1  location =  2 ... ( 2 / 25 )
model1 (DNN with Basis) fit..MSE = 41.314632415771484,,, MAE = 5.221771240234375
model2 (DNN with Covariate) fit..MSE = 59.87761306762695,,, MAE = 6.629988193511963
model_reg (R

# **Results**

In [ ]:
print("MSE_Basis_DNN mean = ", np.mean(MSE_Basis), "/ Std = ", np.std(MSE_Basis))
print("MSE_Covariate_DNN mean = ", np.mean(MSE_Covariate), "/ Std = ", np.std(MSE_Covariate))
print("MSE_Covariate_Reg mean = ", np.mean(MSE_Covariate_Reg), "/ Std = ", np.std(MSE_Covariate_Reg))
print("--------------------------------------")
print("MSE_DNN_DNN mean = ", np.mean(MSE_DNN_DNN), "/ Std = ", np.std(MSE_DNN_DNN))
print("MSE_Reg_DNN mean = ", np.mean(MSE_Reg_DNN), "/ Std = ", np.std(MSE_Reg_DNN))
print("MSE_Reg_Wgt mean = ", np.mean(MSE_Reg_Wgt), "/ Std = ", np.std(MSE_Reg_Wgt))
print("MSE_DNN_Wgt mean = ", np.mean(MSE_DNN_Wgt), "/ Std = ", np.std(MSE_DNN_Wgt))

MSE_Basis_DNN mean =  53.264091415405275 / Std =  26.61851993330698
MSE_Covariate_DNN mean =  77.76350936889648 / Std =  28.10332543929333
MSE_Covariate_Reg mean =  69.75245159150944 / Std =  17.2380682832753
--------------------------------------
MSE_DNN_DNN mean =  39.78470169067383 / Std =  17.76344630568472
MSE_Reg_DNN mean =  40.6794140625 / Std =  17.772579876388473
MSE_Reg_Wgt mean =  40.036140756634644 / Std =  17.02519447389566
MSE_DNN_Wgt mean =  39.2260167647549 / Std =  16.030090635465918


In [ ]:
print("Sqrt_MSE_Basis_DNN mean = ", np.mean(np.sqrt(MSE_Basis))  , '/ Std = ', np.std(np.sqrt(MSE_Basis)) )  
print("Sqrt_MSE_Covariate_DNN mean = ", np.mean(np.sqrt(MSE_Covariate))  , '/ Std = ', np.std(np.sqrt(MSE_Covariate))  )
print("Sqrt_MSE_Covariate_Reg mean = ", np.mean(np.sqrt(MSE_Covariate_Reg))  , '/ Std = ', np.std(np.sqrt(MSE_Covariate_Reg) ) )
print("--------------------------------------")
print("Sqrt_MSE_DNN_DNN mean = ", np.mean(np.sqrt(MSE_DNN_DNN))  , '/ Std = ', np.std(np.sqrt(MSE_DNN_DNN))  )
print("Sqrt_MSE_Reg_DNN mean = ", np.mean(np.sqrt(MSE_Reg_DNN))  , '/ Std = ', np.std(np.sqrt(MSE_Reg_DNN))  )
print("Sqrt_MSE_Reg_Wgt mean = ", np.mean(np.sqrt(MSE_Reg_Wgt))  , '/ Std = ', np.std(np.sqrt(MSE_Reg_Wgt))  )
print("Sqrt_MSE_DNN_Wgt mean = ", np.mean(np.sqrt(MSE_DNN_Wgt))  , '/ Std = ', np.std(np.sqrt(MSE_DNN_Wgt))  )

Sqrt_MSE_Basis_DNN mean =  7.0903937798800225 / Std =  1.729279463141793
Sqrt_MSE_Covariate_DNN mean =  8.694557188747206 / Std =  1.4724756909710848
Sqrt_MSE_Covariate_Reg mean =  8.28455834978699 / Std =  1.0576126608943968
--------------------------------------
Sqrt_MSE_DNN_DNN mean =  6.16307683443463 / Std =  1.3420825695645757
Sqrt_MSE_Reg_DNN mean =  6.232795037795374 / Std =  1.3533957585766847
Sqrt_MSE_Reg_Wgt mean =  6.1910861457878426 / Std =  1.306366369770932
Sqrt_MSE_DNN_Wgt mean =  6.137170178686738 / Std =  1.2494634698914124


# R 결과

**1. Without Covariate**

```

```

---

**2. With Covariate**

```
> acc
 [1]  67.73125  44.30942  61.77689  45.20541  49.44973  64.74137  43.25497  64.91315  40.22579
[10]  29.79341  66.00340  60.69261  61.86075  87.26543  94.62138 104.20433  55.70525 102.67190
[19]  88.77406  62.77618  59.88496  65.88396  71.57981  69.77092  72.30629
> mean(acc)
[1] 65.41611
> sd(acc)
[1] 18.88905
> 
> 
> mean(sqrt(acc))
[1] 8.006119
> sd(sqrt(acc))
[1] 1.171788
```